# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv("../output_data/cities.csv")
vacation_df


,City Data,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Nanortalik,60.1432,-45.2371,41.32,85,35,9.71,GL,2022-08-22 23:07:21
1,1,Avarua,-21.2078,-159.7750,77.05,69,75,12.66,CK,2022-08-22 23:03:33
2,2,Ushuaia,-54.8000,-68.3000,37.06,65,0,5.75,AR,2022-08-22 23:07:21
3,3,Kamaishi,39.2667,141.8833,78.35,85,93,10.07,JP,2022-08-22 23:05:22
4,4,Rikitea,-23.1203,-134.9692,70.48,70,24,14.09,PF,2022-08-22 23:07:22
...,...,...,...,...,...,...,...,...,...,...
562,562,Faya,18.3851,42.4509,72.12,35,88,8.70,SA,2022-08-22 23:11:27
563,563,Dali,25.7000,100.1833,62.04,92,17,1.66,CN,2022-08-22 23:08:55
564,564,São José da Coroa Grande,-8.8978,-35.1478,73.99,87,99,11.45,BR,2022-08-22 23:11:27
565,565,Katubao,5.8892,124.8294,81.07,79,100,1.63,PH,2022-08-22 23:11:28


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
vacation_df = vacation_df.dropna()
weights = vacation_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = vacation_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=max(vacation_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
filtered_cities_df = vacation_df.loc[ (vacation_df["Max Temp"].between(70, 80)) & (vacation_df["Wind Speed"] < 10) & (vacation_df["Cloudiness"] == 0), :]
filtered_cities_df.dropna()
filtered_cities_df

,City Data,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
219,219,Abhā,18.2164,42.5053,71.85,28,0,0.00,SA,2022-08-22 23:08:53
453,453,Mardin,37.3131,40.7436,76.01,40,0,1.25,TR,2022-08-22 23:10:35
475,475,Tucumcari,35.1717,-103.7250,78.64,38,0,6.91,US,2022-08-22 23:10:50
521,521,Saint-Pierre,-21.3393,55.4781,72.91,69,0,8.05,RE,2022-08-22 23:11:10


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = filtered_cities_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 219: Abhā.
Closest hotel is Abha Hotel.


C:\Users\louky\AppData\Local\Temp\ipykernel_13084\2119370261.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.loc[index, 'Hotel Name'] = results[0]['name']


------------
Retrieving Results for Index 453: Mardin.
Closest hotel is Reyhani Kasri Hotel.
------------
Retrieving Results for Index 475: Tucumcari.
Closest hotel is Holiday Inn Express & Suites Tucumcari, an IHG Hotel.
------------
Retrieving Results for Index 521: Saint-Pierre.
Closest hotel is Alize Plage.
------------


In [7]:
hotel_df = hotel_df.dropna()
hotel_df

,City Data,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
219,219,Abhā,18.2164,42.5053,71.85,28,0,0.00,SA,2022-08-22 23:08:53,Abha Hotel
453,453,Mardin,37.3131,40.7436,76.01,40,0,1.25,TR,2022-08-22 23:10:35,Reyhani Kasri Hotel
475,475,Tucumcari,35.1717,-103.7250,78.64,38,0,6.91,US,2022-08-22 23:10:50,"Holiday Inn Express & Suites Tucumcari, an IHG..."
521,521,Saint-Pierre,-21.3393,55.4781,72.91,69,0,8.05,RE,2022-08-22 23:11:10,Alize Plage


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map

 # Assign the marker layer to a variable
    
markers = gmaps.marker_layer(locations=locations, display_info_box=True, info_box_content=hotel_info)

# Add the layer to the map

fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))